In [2]:
import pandas as pd
import numpy as np
from scipy.sparse import load_npz
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss, brier_score_loss

In [3]:
data = load_npz('data/assistments17_first_attempt/X-i.npz')

In [4]:
data

<447587x3167 sparse matrix of type '<class 'numpy.float64'>'
	with 2429253 stored elements in Compressed Sparse Row format>

In [5]:
try:
    for key in data.keys():
        print(key, data[key].shape)
except:
    print("No keys found.")

No keys found.


In [6]:
dense_matrix = data.toarray()

In [7]:
dense_matrix

array([[0.0000000e+00, 1.1170000e+03, 1.0492530e+06, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.1180000e+03, 1.0493020e+06, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.1190000e+03, 1.0493060e+06, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [1.7070000e+03, 1.7110000e+03, 5.2026408e+07, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [1.7070000e+03, 1.7120000e+03, 5.2026420e+07, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [1.7070000e+03, 8.3500000e+02, 5.2026498e+07, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00]])

In [8]:
df = pd.DataFrame(dense_matrix)

In [9]:
df

,0,1,2,3,4,5,6,7,8,9,...,3157,3158,3159,3160,3161,3162,3163,3164,3165,3166
0,0.0,1117.0,1049253.0,0.0,339.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1118.0,1049302.0,1.0,339.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1119.0,1049306.0,0.0,338.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1120.0,1049330.0,1.0,44.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1087.0,1049406.0,0.0,59.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447582,1707.0,1709.0,52026203.0,0.0,158.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
447583,1707.0,1710.0,52026396.0,0.0,158.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
447584,1707.0,1711.0,52026408.0,0.0,158.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
447585,1707.0,1712.0,52026420.0,0.0,158.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
train_df = pd.read_csv(f'data/assistments17_first_attempt/preprocessed_data_train.csv', sep="\t")
test_df = pd.read_csv(f'data/assistments17_first_attempt/preprocessed_data_test.csv', sep="\t")

In [11]:
train_df.head(2)

,user_id,item_id,timestamp,correct,skill_id
0,0,1117,1049253,0,339
1,0,1118,1049302,1,339


In [12]:
# # Student-wise train-test split
# user_ids = X[:, 0].toarray().flatten()
# users_train = train_df["user_id"].unique()
# users_test = test_df["user_id"].unique()
# train = X[np.where(np.isin(user_ids, users_train))]
# test = X[np.where(np.isin(user_ids, users_test))]

In [13]:
# get column 1 of df and remove duplicates
user_ids = df.iloc[:, 0].drop_duplicates()

# get train and test user_id
users_train = train_df["user_id"].unique()
users_test = test_df["user_id"].unique()

In [14]:
# train df where user_id is in users_train
train = df[df.iloc[:, 0].isin(users_train)]

# test df where user_id is in users_test
test = df[df.iloc[:, 0].isin(users_test)]

In [17]:
pd.set_option('display.max_columns', None)
train

0       1           2     3      4     5     6     7     8     9     \
0          0.0  1117.0   1049253.0   0.0  339.0   0.0   0.0   0.0   0.0   0.0   
1          0.0  1118.0   1049302.0   1.0  339.0   0.0   0.0   0.0   0.0   0.0   
2          0.0  1119.0   1049306.0   0.0  338.0   0.0   0.0   0.0   0.0   0.0   
3          0.0  1120.0   1049330.0   1.0   44.0   0.0   0.0   0.0   0.0   0.0   
4          0.0  1087.0   1049406.0   0.0   59.0   0.0   0.0   0.0   0.0   0.0   
...        ...     ...         ...   ...    ...   ...   ...   ...   ...   ...   
447582  1707.0  1709.0  52026203.0   0.0  158.0   0.0   0.0   0.0   0.0   0.0   
447583  1707.0  1710.0  52026396.0   0.0  158.0   0.0   0.0   0.0   0.0   0.0   
447584  1707.0  1711.0  52026408.0   0.0  158.0   0.0   0.0   0.0   0.0   0.0   
447585  1707.0  1712.0  52026420.0   0.0  158.0   0.0   0.0   0.0   0.0   0.0   
447586  1707.0   835.0  52026498.0   1.0  174.0   0.0   0.0   0.0   0.0   0.0   

        10    11    12    13    14    15    16    17    18    19    20    \
0        0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1        0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2        0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3        0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4        0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
...      ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
447582   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
447583   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
447584   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
447585   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
447586   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

        21    22    23    24    25    26    27    28    29    30    31    \
0        0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1        0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2        0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3        0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4        0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
...      ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
447582   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
447583   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
447584   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
447585   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
447586   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

        32    33    34    35    36    37    38    39    40    41    42    \
0        0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1        0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2        0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3        0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4        0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
...      ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
447582   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
447583   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
447584   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
447585   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
447586   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

        43    44    45    46    47    48    49    50    51    52    53    \
0        0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1        0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2        0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0

In [28]:
test

,0,1,2,3,4,5,6,7,8,9,...,3157,3158,3159,3160,3161,3162,3163,3164,3165,3166
2022,7.0,809.0,38976428.0,0.0,70.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2023,7.0,806.0,38976484.0,0.0,340.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024,7.0,807.0,38976873.0,0.0,340.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025,7.0,808.0,38976914.0,1.0,340.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2026,7.0,473.0,38976928.0,0.0,395.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447426,1706.0,201.0,21780256.0,0.0,186.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
447427,1706.0,200.0,21780544.0,0.0,186.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
447428,1706.0,202.0,21780642.0,1.0,186.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
447429,1706.0,368.0,21780699.0,0.0,203.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
# get column 5 onwards of train and test
X_train = train.iloc[:, 5:].to_numpy()
X_test = test.iloc[:, 5:].to_numpy()

# get 4th column of train and test
y_train = train.iloc[:, 3].values.flatten()
y_test = test.iloc[:, 3].values.flatten()

In [46]:
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [47]:
y_train

array([0., 1., 0., ..., 0., 0., 1.])

In [48]:
# Train
model = LogisticRegression(solver="lbfgs", max_iter=100)
model.fit(X_train, y_train)

/opt/anaconda3/envs/suss_task/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [50]:
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

In [53]:
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss, brier_score_loss


In [54]:
accuracy_score(y_test, y_pred_test)


0.6567791932528205

### Dummy curriculum mapping - knowledge components and Q-matrix

In [1]:
import pandas as pd
import numpy as np

In [2]:


sheet1 = pd.read_excel("/Users/harizliewzw/Documents/Adminstrative/Courses/Dummy101/DUMMY101 curriculum_mapping.xlsx", sheet_name=0)
sheet2 = pd.read_excel("/Users/harizliewzw/Documents//Adminstrative/Courses/Dummy101/DUMMY101 curriculum_mapping.xlsx", sheet_name=1) 

In [3]:
# Reverse the multi-hot encoding
def decode_multihot(row):
    return [col for col in sheet2.columns if row[col] == 1]

sheet1['knowledge component'] = sheet2.apply(decode_multihot, axis=1)

In [4]:
# Multi-hot encode the 'Decoded' column
encoded_df = pd.get_dummies(sheet1['knowledge component'].apply(pd.Series).stack()).sum(level=0)

print(encoded_df)

    addition  comparing  counting  division  money  multiplication  patterns  \
0          0          0         1         0      0               0         0   
1          0          0         1         0      0               0         0   
2          0          0         1         0      0               0         0   
3          0          0         0         0      0               0         1   
4          0          1         0         0      0               0         0   
5          1          0         0         0      0               0         0   
6          1          0         0         0      0               0         0   
7          0          0         0         0      0               0         0   
8          0          0         0         0      0               0         0   
9          1          0         0         0      0               0         0   
10         0          0         0         0      0               0         0   
12         0          0         0       

C:\Users\harizliewzw\AppData\Local\Temp\ipykernel_11808\3476277911.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  encoded_df = pd.get_dummies(sheet1['knowledge component'].apply(pd.Series).stack()).sum(level=0)
C:\Users\harizliewzw\AppData\Local\Temp\ipykernel_11808\3476277911.py:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  encoded_df = pd.get_dummies(sheet1['knowledge component'].apply(pd.Series).stack()).sum(level=0)


In [5]:
def get_weights(df):
    df2 = df.iloc[:, 1:] # get just the Kc
    nodes = df2.columns
    arr = df2.values
    m = np.dot(arr.T, arr).astype(float)
    idx = np.tril_indices(m.shape[0])   
    m[idx] = np.nan
    result = pd.DataFrame(m, columns=nodes, index=nodes)
    result = result.stack()
    result = result.astype(int)
    result = result.reset_index()
    result.columns = ['Node A', 'Node B', 'Weights']
    return result

In [6]:
sheet2.head(1)

,Item ID,addition,substraction,multiplication,division,counting,money,comparing,patterns
0,101001,0,0,0,0,1,0,0,0


In [7]:
df = get_weights(sheet2)

In [8]:
df

,Node A,Node B,Weights
0,addition,substraction,3
1,addition,multiplication,1
2,addition,division,2
3,addition,counting,1
4,addition,money,3
5,addition,comparing,0
6,addition,patterns,0
7,substraction,multiplication,2
8,substraction,division,2
9,substraction,counting,1


In [9]:
from pyvis.network import Network

In [15]:
# Create a pyvis network
net = Network(notebook=True, filter_menu=True)

# Extract unique nodes from the DataFrame
unique_nodes = set(df['Node A']).union(set(df['Node B']))

# Add nodes to the network
for node in unique_nodes:
    net.add_node(node)

# Add edges with weights to the network
for index, row in df.iterrows():
    src, dest, weight = row['Node A'], row['Node B'], row['Weights']
    net.add_edge(src, dest, value=weight)
    
# Save the network graph as an HTML file
net.show_buttons()
net.show("network_graph.html")

# Open the HTML file in your web browser
#import webbrowser
# webbrowser.open("network_graph.html")

network_graph.html
